In [ ]:
def get_top_num_order_by(query):
    #import necessary libraries
    from nltk import word_tokenize
    import string
    import re
    import nltk
    import pyodbc
    import inflect
    import pandas as pd
    from textblob import TextBlob
    from difflib import SequenceMatcher

    #define function that will analyse the adjective or adverb selected as the top clause indicator, analyse its polarity and return
    #the order by clause - either 'asc' or 'desc' or ''(in case no order by clause is identified)
    def return_word_sent(word):
        #the flow of the function is as below:
        #1) obtain all synonym sets (as a list) for the top clause identifier using wordnet
        #2) identity the polarity of each synonym in the above list to get a positive or negative score. The synonym is deemed positive if the positive score >negative score
        #3) if more synonyms have a positive polarity than negative, then the top clause identifier is deemed positive.
        #   this usually means the query is aimed at identifying the 'best performing product' or 'the vendor with the highest credit score' etc. Hence the order by clause will likely contain 'desc'
        #   if the top clause identifier is deemed negative then the query is likely to identify 'the product with the lowest sales' etc. so the order by clause will likely contain 'asc'
        
        #additional libraries required for the function.
        from nltk.corpus import sentiwordnet as swn
        from nltk.corpus import wordnet as wn
        #get list of synonym sets for the adjective/adverb selected as the top clause indicator.
        synset_list=wn.synsets(word)
        pos_score=0
        neg_score=0
        #identify if the top clause identifier has an overall positive or negative score, and return order by clause accordingly.
        for synset in synset_list:
            pos=swn.senti_synset(synset.name()).pos_score()
            neg=swn.senti_synset(synset.name()).neg_score()
            if(pos>neg):
                pos_score+=1
            else:
                neg_score+=1
        if(pos_score>neg_score):
            return 'desc'
        elif(neg_score>pos_score):
            return 'asc'
        else:
            return ''
    
    #remove any punctuations from the query
    regex_comp = re.compile('[%s]' % re.escape(string.punctuation))
    query=regex_comp.sub('',query)
    #convert all letters to lower case
    query=query.lower()
    #tokenize the cleaned query
    tok_query=word_tokenize(query)
    #assign part of speech tags to the tokenized query
    pos_query=nltk.pos_tag(tok_query)

    #load tokenized query with pos tags to a dataframe
    df=pd.DataFrame(pos_query,columns=['word','pos_tag'])
    #remove any extra spaces from any words
    df.word=df.word.str.strip()
    #initialize pyodbc connector to connect to the adventure works database
    cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                          "Server=DESKTOP-FPH6QIE\SQLEXPRESS;"
                          "Database=AdventureWorks2012;"
                          "Trusted_Connection=yes;")

    cursor = cnxn.cursor()
    #obtain list of all view column names. These will be used to identify the column which the user query is pointing towards.
    sql_query='select distinct column_name from INFORMATION_SCHEMA.columns where table_name like \'v%\' and column_name not like \'businessentity%\' order by column_name'
    cursor.execute(sql_query)
    rows=cursor.fetchall()
    #load view colunms into a list
    column_list=[]
    for row in rows:
        column_list.append(row[0])
    #of all the nouns in the user query, identify which one matches most closely with the available column names.
    #the noun with the highest match percentage will be treated as the column with the necessary data and hence the subject of the sentence
    mat_ratio=0
    max_match_noun=''
    for noun_word in df.loc[df['pos_tag'].str.contains('NN'),'word']:
        match_per=0
        match_per_t=0
        for column in column_list:
            match_per_t=SequenceMatcher(None, noun_word, column).ratio()
            if(match_per_t>match_per):
                match_per=match_per_t
        if(match_per>mat_ratio):
            mat_ratio=match_per
            max_match_noun=noun_word

    df['row_num']=df.index.values
    subject_row=int(df.loc[df['word']==max_match_noun,'row_num'])
    #If there are multiple adjective/adverbs before the identified subject, then the word closest to the subject is picked.
    df['dist_from_subj']=0
    df.loc[(df['pos_tag']=='JJ')|(df['pos_tag']=='JJS')|(df['pos_tag']=='RB')|(df['pos_tag']=='RBS'),'dist_from_subj']=abs(df['row_num']-subject_row)
    int_df=df.loc[df['dist_from_subj']!=0,['word','dist_from_subj']].sort_values('dist_from_subj')
    int_df=int_df.reset_index()
    top_cl_num=''
    order_by_param=''
    def_rows='5'
    if(len(int_df)>0):
        #pick the adjective/adverb closest to the identified subject as the top clause identifier.
        top_cl_word=int_df.loc[0,'word']
        order_by_param=''
        top_cl_ident=df[['word','pos_tag']].copy()
        #the top clause is accompanied by a number, which is usually either after the clause or before the clause:
        # return top 10 highest selling products
        # return 10 highest selling products
        # the code below attempts to identify this number, if it is present in the query.
        top_cl_ident['prev_index']=top_cl_ident.index.values-1
        top_cl_ident['next_index']=top_cl_ident.index.values+1
        top_cl_ident=pd.merge(top_cl_ident[['word','pos_tag','next_index','prev_index']],top_cl_ident[['word','pos_tag']],how='left',left_on='prev_index',right_index=True,suffixes=('','_prev'))
        top_cl_ident=pd.merge(top_cl_ident[['word','pos_tag','next_index','word_prev','pos_tag_prev']],top_cl_ident[['word','pos_tag']],how='left',left_on='next_index',right_index=True,suffixes=('','_next'))
        top_cl_ident=top_cl_ident.loc[(top_cl_ident['word']==top_cl_word),['word','pos_tag','word_prev','pos_tag_prev','word_next','pos_tag_next']]
        top_cl_ident=top_cl_ident.reset_index()

        noun_string=top_cl_ident.loc[0,'word']
        for word in df.loc[df['pos_tag'].str.contains('NN'),'word']:
            noun_string=noun_string+' '+word
    
        noun_str_polarity=TextBlob(noun_string).sentiment.polarity
        if(noun_str_polarity >0):
            order_by_param='desc'
        elif(noun_str_polarity<0):
            order_by_param='asc'
        else:
            order_by_param=return_word_sent(top_cl_ident['word'].values[0])
    
        if(top_cl_ident.loc[0,'pos_tag_prev']=='CD' and top_cl_ident.loc[0,'pos_tag_next']!='CD'):
            top_cl_num=str(top_cl_ident.loc[0,'word_prev'])
        elif(top_cl_ident.loc[0,'pos_tag_prev']!='CD' and top_cl_ident.loc[0,'pos_tag_next']=='CD'):
            top_cl_num=str(top_cl_ident.loc[0,'word_next'])
        else:
            if(str(df.loc[(df['word']==max_match_noun),'pos_tag'].values[0])=='NN' or str(df.loc[(df['word']==max_match_noun),'pos_tag'].values[0])=='NNP'):
                top_cl_num='1'
            else:
                top_cl_num=def_rows
            
    return(top_cl_num,order_by_param)
